# Torch bridges examples

In [1]:
import pickle

from datasets import load_dataset
from plaid.bridges.huggingface_bridge import (
    huggingface_dataset_to_plaid,
    huggingface_description_to_problem_definition,
)
from plaid.containers.sample import Sample

from torch.utils.data import DataLoader

from plaid_bridges.torch.base import HomogeneousDataset
from plaid_bridges.torch.grid import GridFieldsAndScalarsDataset
import copy
import numpy as np

/home/fabien/miniconda3/envs/plaid-bridges/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
hf_dataset = load_dataset("PLAID-datasets/VKI-LS59", split="all_samples[:10]")
sample = Sample.model_validate(pickle.loads(hf_dataset[0]["sample"]))

pb_def = huggingface_description_to_problem_definition(hf_dataset.info.description)
ids = pb_def.get_split("train")[:10]

dataset, _ = huggingface_dataset_to_plaid(hf_dataset, ids=ids, processes_number=5)

Converting huggingface dataset to plaid dataset...


100%|██████████| 10/10 [00:00<00:00, 76.34it/s]


In [3]:
print(dataset)
all_feat_ids = dataset[0].get_all_features_identifiers()

scalar_features = [f for f in all_feat_ids if "scalar" in f.values()]
field_features = [f for f in all_feat_ids if "field" in f.values() and "Base_2_2" in f.values() ]

in_feature_identifiers = scalar_features
out_feature_identifiers = field_features

print(in_feature_identifiers)
print(out_feature_identifiers)

Dataset(10 samples, 8 scalars, 0 time_series, 8 fields)
[{'type': 'scalar', 'name': np.str_('Pr')}, {'type': 'scalar', 'name': np.str_('Q')}, {'type': 'scalar', 'name': np.str_('Tr')}, {'type': 'scalar', 'name': np.str_('angle_in')}, {'type': 'scalar', 'name': np.str_('angle_out')}, {'type': 'scalar', 'name': np.str_('eth_is')}, {'type': 'scalar', 'name': np.str_('mach_out')}, {'type': 'scalar', 'name': np.str_('power')}]
[{'type': 'field', 'name': 'roe', 'base_name': 'Base_2_2', 'zone_name': 'Zone', 'location': 'Vertex', 'time': np.float64(0.0)}, {'type': 'field', 'name': 'nut', 'base_name': 'Base_2_2', 'zone_name': 'Zone', 'location': 'Vertex', 'time': np.float64(0.0)}, {'type': 'field', 'name': 'rou', 'base_name': 'Base_2_2', 'zone_name': 'Zone', 'location': 'Vertex', 'time': np.float64(0.0)}, {'type': 'field', 'name': 'ro', 'base_name': 'Base_2_2', 'zone_name': 'Zone', 'location': 'Vertex', 'time': np.float64(0.0)}, {'type': 'field', 'name': 'sdf', 'base_name': 'Base_2_2', 'zone_na

In [4]:
homogen_dataset = HomogeneousDataset(
    dataset = dataset,
    in_feature_identifiers = in_feature_identifiers,
    out_feature_identifiers = out_feature_identifiers,
)

homogen_dataset.show_details()

RegressionDataset (10 sample, 8 input features, 7) output features)
Input features : ['Pr (scalar)', 'Q (scalar)', 'Tr (scalar)', 'angle_in (scalar)', 'angle_out (scalar)', 'eth_is (scalar)', 'mach_out (scalar)', 'power (scalar)']
Output features: ['roe (field)', 'nut (field)', 'rou (field)', 'ro (field)', 'sdf (field)', 'mach (field)', 'rov (field)']


In [5]:
loader = DataLoader(
    homogen_dataset,
    batch_size=2,
    shuffle=False,
)

out_feat_id = out_feature_identifiers[0]
before = copy.deepcopy(dataset[ids[1]].get_feature_from_identifier(out_feat_id))

predictions = []
for batch in loader:
    for torch_sample in batch:
        predictions.append(torch_sample[1])

dataset_pred = homogen_dataset.inverse_transform(predictions)

after = copy.deepcopy(dataset_pred[ids[1]].get_feature_from_identifier(out_feat_id))

print("Error after transform then inverse transform (2nd sample):")
print(np.linalg.norm(after - before)/np.linalg.norm(before))

Error after transform then inverse transform (2nd sample):
0.0
